In [1]:
%env RAY_verbose_spill_logs=0
%env RAY_SESSION_DIR="/home/ray/"

env: RAY_verbose_spill_logs=0
env: RAY_SESSION_DIR="/home/ray/"


In [2]:
import ray
import os
import boto3
from concurrent.futures import ThreadPoolExecutor

# Number of workers to use
num_loops = 10

# Number to calculate sum of squares up to
N = 100

# Shared storage configuration
s3_endpoint = "https://home.hpe-apps-ezaf.com:31900"  # MinIO endpoint
s3_access_key = "minioadmin"
s3_secret_key = "minioadmin"
s3_bucket = "experiments"

# Create an S3 client
clientArgs = {
        'aws_access_key_id': 'minioadmin',
        'aws_secret_access_key': 'minioadmin',
        'endpoint_url': 'https://home.hpe-apps-ezaf.com:31900',
        'verify': False
        }
s3_client = boto3.client("s3", **clientArgs)
print("s3_client:", s3_client)

# Initialize the RAY cluster
ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True)

# Define the function that calculates the sum of squares
def calculate_sum_of_squares(n):
    return sum(i * i for i in range(1, n + 1))

# Function to distribute the computation and write results
def distributed_task(worker_id, n):
    try:
        # Calculate the result
        result = calculate_sum_of_squares(n)
        
        # Save the result to a local file
        result_file = f"result_{worker_id}.txt"
        with open(result_file, "w") as f:
            f.write(str(result))
        
        # Upload the result file to S3
        # s3_key = f"{s3_bucket}/{result_file}"
        s3_key = f"{result_file}"
        print("result_file:", result_file, "||", "s3_bucket:", s3_bucket, "||", "s3_key:", s3_key)
        
        with open(f"{result_file}", 'rb') as data:
            s3_client.upload_fileobj(data, s3_bucket, s3_key)   
        
        return s3_key
    except Exception as e:
        return str(e)

# Launch distributed tasks using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_loops) as executor:
    task_futures = [executor.submit(distributed_task, i, N) for i in range(num_loops)]
    print("task_futures:", list(task_futures))

print("CUSTOM S3 PATH JOB ENDED")

# Shutdown the RAY cluster
ray.shutdown()


2023-08-16 12:18:30,390	INFO client_builder.py:237 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


s3_client: <botocore.client.S3 object at 0x7fb4d6403190>
task_futures: [<Future at 0x7fb4c84d68e0 state=running>, <Future at 0x7fb4c84e8670 state=running>, <Future at 0x7fb4c84e8cd0 state=running>, <Future at 0x7fb4c8475370 state=running>, <Future at 0x7fb4c8475760 state=running>, <Future at 0x7fb4c8475b50 state=running>, <Future at 0x7fb4c8475f40 state=running>, <Future at 0x7fb4c847b370 state=running>, <Future at 0x7fb4c847b760 state=running>, <Future at 0x7fb4c847bb50 state=running>]
result_file: result_0.txt || s3_bucket: experiments || s3_key: result_0.txt
result_file: result_2.txt || s3_bucket: experiments || s3_key: result_2.txt
result_file: result_1.txt || s3_bucket: experiments || s3_key: result_1.txt
result_file: result_4.txt || s3_bucket: experiments || s3_key: result_4.txt
result_file: result_3.txt || s3_bucket: experiments || s3_key: result_3.txt
result_file: result_5.txt || s3_bucket: experiments || s3_key: result_5.txt
result_file: result_6.txt || s3_bucket: experiments 

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'home.hpe-apps-ezaf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'home.hpe-apps-ezaf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'home.hpe-apps-ezaf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/co

CUSTOM S3 PATH JOB ENDED
